https://towardsdatascience.com/state-of-the-art-machine-learning-hyperparameter-optimization-with-optuna-a315d8564de1

In [ ]:
!pip install optuna

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 365.7/365.7 kB 11.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.3/212.3 kB 18.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.7/78.7 kB 7.9 MB/s eta 0:00:00


In [ ]:
import optuna
import sklearn
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score
def objective(trial):
    digits = sklearn.datasets.load_digits()
    x, y = digits.data, digits.target
    max_depth = trial.suggest_int("rf_max_depth", 2, 64, log=True)
    max_samples = trial.suggest_float("rf_max_samples", 0.2, 1)
   
    rf_model = RandomForestClassifier(
        max_depth = max_depth,
        max_samples = max_samples,
        n_estimators = 50,
        random_state = 42)
    
    score = cross_val_score(rf_model, x, y,  cv=3).mean()
    
    return score

In [ ]:
study = optuna.create_study(direction = "maximize")
study.optimize(objective, n_trials = 100)
trial = study.best_trial
print("Best Score: ", trial.value)
print("Best Params: ")
for key, value in trial.params.items():
    print("  {}: {}".format(key, value))

[I 2023-04-18 09:06:00,907] A new study created in memory with name: no-name-255de14c-40ac-4483-9148-ffc4e3eafc7d
[I 2023-04-18 09:06:01,555] Trial 0 finished with value: 0.7824151363383417 and parameters: {'rf_max_depth': 2, 'rf_max_samples': 0.5492884970974976}. Best is trial 0 with value: 0.7824151363383417.
[I 2023-04-18 09:06:02,321] Trial 1 finished with value: 0.9087367835281025 and parameters: {'rf_max_depth': 6, 'rf_max_samples': 0.577258456082899}. Best is trial 1 with value: 0.9087367835281025.
[I 2023-04-18 09:06:02,960] Trial 2 finished with value: 0.8386199220923762 and parameters: {'rf_max_depth': 3, 'rf_max_samples': 0.36924538351232816}. Best is trial 1 with value: 0.9087367835281025.
[I 2023-04-18 09:06:03,669] Trial 3 finished with value: 0.9282136894824707 and parameters: {'rf_max_depth': 12, 'rf_max_samples': 0.2973027711500656}. Best is trial 3 with value: 0.9282136894824707.
[I 2023-04-18 09:06:04,326] Trial 4 finished with value: 0.9326655537006121 and parameter

Best Score:  0.9465776293823037
Best Params: 
  rf_max_depth: 40
  rf_max_samples: 0.8404196430711574


In [ ]:
optuna.visualization.plot_contour(study)

In [ ]:
optuna.visualization.plot_param_importances(study)

In [ ]:
import optuna
from optuna.pruners import SuccessiveHalvingPruner
from optuna.samplers import TPESampler
from sklearn.model_selection import train_test_split
from sklearn.datasets import load_iris
from sklearn.linear_model import SGDClassifier
def objective(trial):
# Loading the data set and splitting into train, test sets
    iris = load_iris()
    classes = list(set(iris.target))
    train_x, valid_x, train_y, valid_y = train_test_split(
        iris.data, iris.target, test_size=0.25
    )
    # Prompting Optuna to suggest a hyperparameter value
    alpha = trial.suggest_float("alpha", 1e-5, 1e-1, log=True)

    sgd = SGDClassifier(alpha = alpha, random_state = 42)

    # Report the intermediate score for every step
    for step in range(100):
        sgd.partial_fit(train_x, train_y, classes=classes)
        # Report the intermediate objective value.
        intermediate_value = sgd.score(valid_x, valid_y)
        trial.report(intermediate_value, step)
        # Prune the intermediate value if neccessary.
        if trial.should_prune():
            raise optuna.TrialPruned()
    return sgd.score(valid_x, valid_y)

In [ ]:
study = optuna.create_study(sampler = TPESampler(), 
                            pruner = SuccessiveHalvingPruner(),
                            direction= "maximize")

study.optimize(objective, n_trials = 25)

pruned_trials = study.get_trials(states=[optuna.trial.TrialState.PRUNED])

complete_trials = study.get_trials(states=[optuna.trial.TrialState.COMPLETE])

print("# Pruned trials: ", len(pruned_trials))
print("# Complete trials: ", len(complete_trials))

trial = study.best_trial

print("Best Score: ", trial.value)

print("Best Params: ")
for key, value in trial.params.items():
    print("  {}: {}".format(key, value))

[I 2023-04-18 08:43:03,281] A new study created in memory with name: no-name-951e697a-2edd-4b99-a353-960a05c68498


<class 'optuna.trial._trial.Trial'> <optuna.trial._trial.Trial object at 0x7f4d95646e80>


[I 2023-04-18 08:43:03,585] Trial 0 finished with value: 0.9210526315789473 and parameters: {'alpha': 0.07629906581862377}. Best is trial 0 with value: 0.9210526315789473.


<class 'optuna.trial._trial.Trial'> <optuna.trial._trial.Trial object at 0x7f4d95646e80>


[I 2023-04-18 08:43:03,866] Trial 1 finished with value: 0.6842105263157895 and parameters: {'alpha': 3.836601635639834e-05}. Best is trial 0 with value: 0.9210526315789473.
[I 2023-04-18 08:43:03,932] Trial 2 pruned. 
[I 2023-04-18 08:43:03,946] Trial 3 pruned. 
[I 2023-04-18 08:43:03,958] Trial 4 pruned. 
[I 2023-04-18 08:43:03,972] Trial 5 pruned. 
[I 2023-04-18 08:43:04,029] Trial 6 pruned. 
[I 2023-04-18 08:43:04,050] Trial 7 pruned. 
[I 2023-04-18 08:43:04,072] Trial 8 pruned. 


...

<class 'optuna.trial._trial.Trial'> <optuna.trial._trial.Trial object at 0x7f4d95641730>
<class 'optuna.trial._trial.Trial'> <optuna.trial._trial.Trial object at 0x7f4d95e3a0d0>
<class 'optuna.trial._trial.Trial'> <optuna.trial._trial.Trial object at 0x7f4d95660790>
<class 'optuna.trial._trial.Trial'> <optuna.trial._trial.Trial object at 0x7f4d956a1760>
<class 'optuna.trial._trial.Trial'> <optuna.trial._trial.Trial object at 0x7f4da70082e0>
<class 'optuna.trial._trial.Trial'> <optuna.trial._trial.Trial object at 0x7f4d95642040>
<class 'optuna.trial._trial.Trial'> <optuna.trial._trial.Trial object at 0x7f4d955e5b80>
<class 'optuna.trial._trial.Trial'> <optuna.trial._trial.Trial object at 0x7f4d95646d60>


[I 2023-04-18 08:43:04,332] Trial 17 pruned. 
[I 2023-04-18 08:43:04,353] Trial 18 pruned. 
[I 2023-04-18 08:43:04,374] Trial 19 pruned. 
[I 2023-04-18 08:43:04,400] Trial 20 pruned. 


<class 'optuna.trial._trial.Trial'> <optuna.trial._trial.Trial object at 0x7f4d956460a0>
<class 'optuna.trial._trial.Trial'> <optuna.trial._trial.Trial object at 0x7f4d95e7c580>
<class 'optuna.trial._trial.Trial'> <optuna.trial._trial.Trial object at 0x7f4d9566b370>
<class 'optuna.trial._trial.Trial'> <optuna.trial._trial.Trial object at 0x7f4d9562ad90>


[I 2023-04-18 08:43:04,694] Trial 21 finished with value: 0.8947368421052632 and parameters: {'alpha': 0.010563497350391327}. Best is trial 0 with value: 0.9210526315789473.
[I 2023-04-18 08:43:04,720] Trial 22 pruned. 
[I 2023-04-18 08:43:04,736] Trial 23 pruned. 
[I 2023-04-18 08:43:04,752] Trial 24 pruned. 


<class 'optuna.trial._trial.Trial'> <optuna.trial._trial.Trial object at 0x7f4d95e39c40>
<class 'optuna.trial._trial.Trial'> <optuna.trial._trial.Trial object at 0x7f4d95ea5820>
<class 'optuna.trial._trial.Trial'> <optuna.trial._trial.Trial object at 0x7f4d95ee04f0>
# Pruned trials:  22
# Complete trials:  3
Best Score:  0.9210526315789473
Best Params: 
  alpha: 0.07629906581862377


In [ ]:
optuna.visualization.plot_optimization_history(study)

In [ ]:
optuna.visualization.plot_intermediate_values(study)

16
